In [1]:
!pip install spectral

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import keras
import tensorflow as tf
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization
from keras.layers import Dropout, Input
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score

from operator import truediv

from plotly.offline import init_notebook_mode

import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import spectral

init_notebook_mode(connected=True)
%matplotlib inline

In [3]:
if not (os.path.isfile('Indian_pines_corrected.mat')):
  !wget http://www.ehu.eus/ccwintco/uploads/6/67/Indian_pines_corrected.mat
if not (os.path.isfile('Indian_pines_gt.mat')):
  !wget http://www.ehu.eus/ccwintco/uploads/c/c4/Indian_pines_gt.mat

#if not (os.path.isfile('/content/Salinas_corrected.mat')):
#  !wget https://github.com/gokriznastic/HybridSN/raw/master/data/Salinas_corrected.mat
#if not (os.path.isfile('/content/Salinas_gt.mat')):
#  !wget https://github.com/gokriznastic/HybridSN/raw/master/data/Salinas_gt.mat

# Data Loading

In [4]:
## GLOBAL VARIABLES
dataset = 'PU'
test_ratio = 0.5
windowSize = 25

In [5]:
def loadData(name):
    data_path = os.path.join(os.getcwd(),'data')
    if name == 'IP':
        data = sio.loadmat(os.path.join(data_path, 'Indian_pines_corrected.mat'))['indian_pines_corrected']
        labels = sio.loadmat(os.path.join(data_path, 'Indian_pines_gt.mat'))['indian_pines_gt']
    elif name == 'SA':
        data = sio.loadmat(os.path.join(data_path, 'Salinas_corrected.mat'))['salinas_corrected']
        labels = sio.loadmat(os.path.join(data_path, 'Salinas_gt.mat'))['salinas_gt']
    elif name == 'PU':
        data = sio.loadmat(os.path.join(data_path, 'PaviaU.mat'))['paviaU']
        labels = sio.loadmat(os.path.join(data_path, 'PaviaU_gt.mat'))['paviaU_gt']
    
    return data, labels

In [6]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test

In [7]:
def applyPCA(X, numComponents=75):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX, pca

In [8]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

In [9]:
def createImageCubes(X, y, windowSize=5, removeZeroLabels = True):
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin + 1, c - margin:c + margin + 1]   
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [10]:
X, y = loadData(dataset)

X.shape, y.shape

((610, 340, 103), (610, 340))

In [11]:
K = X.shape[2]

In [12]:
K = 30 if dataset == 'IP' else 15
X,pca = applyPCA(X,numComponents=K)

X.shape

(610, 340, 15)

In [13]:
X, y = createImageCubes(X, y, windowSize=windowSize)

X.shape, y.shape

((42776, 25, 25, 15), (42776,))

In [14]:
Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X, y, test_ratio)

Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape

((21388, 25, 25, 15), (21388, 25, 25, 15), (21388,), (21388,))

Xtrain, Xvalid, ytrain, yvalid = splitTrainTestSet(Xtrain, ytrain, 0.3333)

Xtrain.shape, Xvalid.shape, ytrain.shape, yvalid.shape

# Model and Training

In [15]:
Xtrain = Xtrain.reshape(-1, windowSize, windowSize, K, 1)
Xtrain.shape

(21388, 25, 25, 15, 1)

In [16]:
ytrain = np_utils.to_categorical(ytrain)
ytrain.shape

(21388, 9)

Xvalid = Xvalid.reshape(-1, windowSize, windowSize, K, 1)
Xvalid.shape

yvalid = np_utils.to_categorical(yvalid)
yvalid.shape

In [17]:
S = windowSize
L = K
output_units = 9 if (dataset == 'PU' or dataset == 'PC') else 16

In [18]:
## input layer
input_layer = Input((S, S, L, 1))

## convolutional layers
conv_layer1 = Conv3D(filters=8, kernel_size=(3, 3, 7), activation='relu')(input_layer)
conv_layer2 = Conv3D(filters=16, kernel_size=(3, 3, 5), activation='relu')(conv_layer1)
conv_layer3 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(conv_layer2)
#print(conv_layer3._keras_shape)
conv3d_shape = conv_layer3.shape
conv_layer3 = Reshape((conv3d_shape[1], conv3d_shape[2], conv3d_shape[3]*conv3d_shape[4]))(conv_layer3)
conv_layer4 = Conv2D(filters=64, kernel_size=(3,3), activation='relu')(conv_layer3)

flatten_layer = Flatten()(conv_layer4)

## fully connected layers
dense_layer1 = Dense(units=256, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=128, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=output_units, activation='softmax')(dense_layer2)

2023-05-15 22:35:34.815678: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-15 22:35:34.862738: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-15 22:35:34.863037: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-15 22:35:34.889446: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-15 22:35:34.890001: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

In [19]:
# define the model with input layer and output layer
model = Model(inputs=input_layer, outputs=output_layer)

In [20]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 25, 25, 15, 1)]   0         
                                                                 
 conv3d (Conv3D)             (None, 23, 23, 9, 8)      512       
                                                                 
 conv3d_1 (Conv3D)           (None, 21, 21, 5, 16)     5776      
                                                                 
 conv3d_2 (Conv3D)           (None, 19, 19, 3, 32)     13856     
                                                                 
 reshape (Reshape)           (None, 19, 19, 96)        0         
                                                                 
 conv2d (Conv2D)             (None, 17, 17, 64)        55360     
                                                                 
 flatten (Flatten)           (None, 18496)             0     

In [21]:
# compiling the model
adam = Adam(learning_rate=0.001, decay=1e-06)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [22]:
# checkpoint
filepath = "best-model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [23]:
history = model.fit(x=Xtrain, y=ytrain, batch_size=128, epochs=100, callbacks=callbacks_list)

Epoch 1/100


2023-05-15 22:35:41.255637: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8500
2023-05-15 22:35:42.062421: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-05-15 22:35:42.193522: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


168/168 [==============================] - 7s 18ms/step - loss: 0.4413 - accuracy: 0.8501
Epoch 2/100
168/168 [==============================] - 3s 18ms/step - loss: 0.0330 - accuracy: 0.9909
Epoch 3/100
168/168 [==============================] - 3s 18ms/step - loss: 0.0191 - accuracy: 0.9949
Epoch 4/100
168/168 [==============================] - 3s 16ms/step - loss: 0.0145 - accuracy: 0.9962
Epoch 5/100
168/168 [==============================] - 3s 18ms/step - loss: 0.0053 - accuracy: 0.9985
Epoch 6/100
168/168 [==============================] - 3s 18ms/step - loss: 0.0317 - accuracy: 0.9921
Epoch 7/100
168/168 [==============================] - 3s 18ms/step - loss: 0.0110 - accuracy: 0.9977
Epoch 8/100
168/168 [==============================] - 3s 18ms/step - loss: 0.0167 - accuracy: 0.9961
Epoch 9/100
168/168 [==============================] - 3s 18ms/step - loss: 0.0068 - accuracy: 0.9982
Epoch 10/100
168/168 [==============================] - 3s 18ms/step - loss: 0.0022 - accuracy

168/168 [==============================] - 3s 17ms/step - loss: 0.0029 - accuracy: 0.9996
Epoch 33/100
168/168 [==============================] - 3s 17ms/step - loss: 0.0043 - accuracy: 0.9994
Epoch 34/100
168/168 [==============================] - 3s 17ms/step - loss: 0.0016 - accuracy: 0.9997
Epoch 35/100
168/168 [==============================] - 3s 17ms/step - loss: 9.6570e-04 - accuracy: 0.9998
Epoch 36/100
168/168 [==============================] - 3s 17ms/step - loss: 0.0143 - accuracy: 0.9982
Epoch 37/100
168/168 [==============================] - 3s 17ms/step - loss: 0.0283 - accuracy: 0.9957
Epoch 38/100
168/168 [==============================] - 3s 17ms/step - loss: 0.0179 - accuracy: 0.9976
Epoch 39/100
168/168 [==============================] - 3s 16ms/step - loss: 0.0134 - accuracy: 0.9979
Epoch 40/100
168/168 [==============================] - 3s 17ms/step - loss: 0.0067 - accuracy: 0.9987
Epoch 41/100
168/168 [==============================] - 3s 16ms/step - loss: 0.008

Epoch 64/100
168/168 [==============================] - 3s 17ms/step - loss: 0.0037 - accuracy: 0.9994
Epoch 65/100
168/168 [==============================] - 3s 17ms/step - loss: 0.0038 - accuracy: 0.9996
Epoch 66/100
168/168 [==============================] - 3s 17ms/step - loss: 0.0070 - accuracy: 0.9993
Epoch 67/100
168/168 [==============================] - 3s 17ms/step - loss: 0.0043 - accuracy: 0.9994
Epoch 68/100
168/168 [==============================] - 3s 17ms/step - loss: 0.0117 - accuracy: 0.9992
Epoch 69/100
168/168 [==============================] - 3s 17ms/step - loss: 0.0286 - accuracy: 0.9973
Epoch 70/100
168/168 [==============================] - 3s 17ms/step - loss: 0.0151 - accuracy: 0.9990
Epoch 71/100
168/168 [==============================] - 3s 17ms/step - loss: 0.0092 - accuracy: 0.9991
Epoch 72/100
168/168 [==============================] - 3s 17ms/step - loss: 0.0016 - accuracy: 0.9998
Epoch 73/100
168/168 [==============================] - 3s 17ms/step - lo

168/168 [==============================] - 3s 17ms/step - loss: 1.7674e-04 - accuracy: 0.9999
Epoch 96/100
168/168 [==============================] - 3s 18ms/step - loss: 0.0047 - accuracy: 0.9998
Epoch 97/100
168/168 [==============================] - 3s 16ms/step - loss: 0.0102 - accuracy: 0.9993
Epoch 98/100
168/168 [==============================] - 3s 17ms/step - loss: 0.0065 - accuracy: 0.9997
Epoch 99/100
168/168 [==============================] - 3s 17ms/step - loss: 0.0068 - accuracy: 0.9995
Epoch 100/100
168/168 [==============================] - 3s 17ms/step - loss: 0.0296 - accuracy: 0.9986


In [24]:
model.save("best-model.hdf5")

# Validation

In [25]:
# load best weights
model.load_weights("best-model.hdf5")
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [26]:
Xtest = Xtest.reshape(-1, windowSize, windowSize, K, 1)
Xtest.shape

(21388, 25, 25, 15, 1)

In [27]:
ytest = np_utils.to_categorical(ytest)
ytest.shape

(21388, 9)

In [28]:
Y_pred_test = model.predict(Xtest)
y_pred_test = np.argmax(Y_pred_test, axis=1)

classification = classification_report(np.argmax(ytest, axis=1), y_pred_test)
print(classification)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3316
           1       1.00      1.00      1.00      9324
           2       1.00      1.00      1.00      1049
           3       1.00      1.00      1.00      1532
           4       1.00      1.00      1.00       673
           5       1.00      1.00      1.00      2514
           6       1.00      1.00      1.00       665
           7       1.00      1.00      1.00      1841
           8       1.00      1.00      1.00       474

    accuracy                           1.00     21388
   macro avg       1.00      1.00      1.00     21388
weighted avg       1.00      1.00      1.00     21388



In [29]:
def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

In [30]:
def reports (X_test,y_test,name):
    #start = time.time()
    Y_pred = model.predict(X_test)
    y_pred = np.argmax(Y_pred, axis=1)
    #end = time.time()
    #print(end - start)
    if name == 'IP':
        target_names = ['Alfalfa', 'Corn-notill', 'Corn-mintill', 'Corn'
                        ,'Grass-pasture', 'Grass-trees', 'Grass-pasture-mowed', 
                        'Hay-windrowed', 'Oats', 'Soybean-notill', 'Soybean-mintill',
                        'Soybean-clean', 'Wheat', 'Woods', 'Buildings-Grass-Trees-Drives',
                        'Stone-Steel-Towers']
    elif name == 'SA':
        target_names = ['Brocoli_green_weeds_1','Brocoli_green_weeds_2','Fallow','Fallow_rough_plow','Fallow_smooth',
                        'Stubble','Celery','Grapes_untrained','Soil_vinyard_develop','Corn_senesced_green_weeds',
                        'Lettuce_romaine_4wk','Lettuce_romaine_5wk','Lettuce_romaine_6wk','Lettuce_romaine_7wk',
                        'Vinyard_untrained','Vinyard_vertical_trellis']
    elif name == 'PU':
        target_names = ['Asphalt','Meadows','Gravel','Trees', 'Painted metal sheets','Bare Soil','Bitumen',
                        'Self-Blocking Bricks','Shadows']
    
    classification = classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names)
    oa = accuracy_score(np.argmax(y_test, axis=1), y_pred)
    confusion = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
    each_acc, aa = AA_andEachClassAccuracy(confusion)
    kappa = cohen_kappa_score(np.argmax(y_test, axis=1), y_pred)
    score = model.evaluate(X_test, y_test, batch_size=32)
    Test_Loss =  score[0]*100
    Test_accuracy = score[1]*100
    
    return classification, confusion, Test_Loss, Test_accuracy, oa*100, each_acc*100, aa*100, kappa*100

In [31]:
classification, confusion, Test_loss, Test_accuracy, oa, each_acc, aa, kappa = reports(Xtest,ytest,dataset)
classification = str(classification)
confusion = str(confusion)
file_name = "classification_report_PU.txt"

with open(file_name, 'w') as x_file:
    x_file.write('{} Test loss (%)'.format(Test_loss))
    x_file.write('\n')
    x_file.write('{} Test accuracy (%)'.format(Test_accuracy))
    x_file.write('\n')
    x_file.write('\n')
    x_file.write('{} Kappa accuracy (%)'.format(kappa))
    x_file.write('\n')
    x_file.write('{} Overall accuracy (%)'.format(oa))
    x_file.write('\n')
    x_file.write('{} Average accuracy (%)'.format(aa))
    x_file.write('\n')
    x_file.write('\n')
    x_file.write('{}'.format(classification))
    x_file.write('\n')
    x_file.write('{}'.format(confusion))

669/669 [==============================] - 5s 7ms/step - loss: 0.0135 - accuracy: 0.9996


In [32]:
def Patch(data,height_index,width_index):
    height_slice = slice(height_index, height_index+PATCH_SIZE)
    width_slice = slice(width_index, width_index+PATCH_SIZE)
    patch = data[height_slice, width_slice, :]
    
    return patch

In [33]:
# load the original image
X, y = loadData(dataset)

In [34]:
height = y.shape[0]
width = y.shape[1]
PATCH_SIZE = windowSize
numComponents = K

In [35]:
X,pca = applyPCA(X, numComponents=numComponents)

In [36]:
X = padWithZeros(X, PATCH_SIZE//2)

In [37]:
# calculate the predicted image
outputs = np.zeros((height,width))
for i in range(height):
    for j in range(width):
        target = int(y[i,j])
        if target == 0 :
            continue
        else :
            image_patch=Patch(X,i,j)
            X_test_image = image_patch.reshape(1,image_patch.shape[0],image_patch.shape[1], image_patch.shape[2], 1).astype('float32')                                   
            prediction = (model.predict(X_test_image))
            prediction = np.argmax(prediction, axis=1)
            outputs[i][j] = prediction+1

KeyboardInterrupt: 

In [ ]:
ground_truth = spectral.imshow(classes = y,figsize =(7,7))

In [ ]:
predict_image = spectral.imshow(classes = outputs.astype(int),figsize =(7,7))

In [ ]:
spectral.save_rgb("predictions.jpg", outputs.astype(int), colors=spectral.spy_colors)

spectral.save_rgb(str(dataset)+"_ground_truth.jpg", y, colors=spectral.spy_colors)